In [11]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split

OSError: dlopen(/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/lightgbm/lib/lib_lightgbm.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib
  Referenced from: <D3923ACB-D836-32D3-A031-CF91999FDAFC> /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/lightgbm/lib/lib_lightgbm.dylib
  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file)

In [5]:
file_location = "/Users/alyssonamaral/Documents/ifood-case/case_data/df_modelo.csv"
df = pd.read_csv(file_location)

In [6]:
df.head()


,last_event,account_id,amount,offer_id,target,age,credit_card_limit,registered_on,discount_value,duration,min_value,channel_web,channel_email,channel_mobile,channel_social,offer_bogo,offer_informational,offer_discount,gender_numeric
0,offer received,00bc983061d3471e8c8e74d31b7c8b6f,21.40,2906b810c7d4411798c6938adc9daaa5,0,59,77000.0,20171016,2,7.0,10,1,1,1,0,0,0,1,2
1,offer received,016871ea865d4338975026ae08d221d0,19.04,fafdcd668e3743c1bb461111dcafc2a4,0,57,79000.0,20180601,2,10.0,10,1,1,1,1,0,0,1,1
2,offer received,034dcdec0a0547a19d77dcf2bc8ecc48,11.37,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,57,76000.0,20171001,5,7.0,5,1,1,1,0,1,0,0,1
3,offer received,03e866224ec641c58bc93a69be00de9c,4.23,2906b810c7d4411798c6938adc9daaa5,0,80,47000.0,20180211,2,7.0,10,1,1,1,0,0,0,1,1
4,offer received,05ea8d8a95eb4b209a6e048878470df1,36.23,ae264e3637204a6fb9bb56bc8210ddfd,0,62,111000.0,20151103,10,7.0,10,0,1,1,1,1,0,0,1


In [ ]:
df['group_id'] = df.groupby('account_id').ngroup()

# Definir features e target
features = ['age', 'credit_card_limit', 'gender_numeric',
            'offer_bogo', 'offer_discount', 'offer_informational',
            'channel_web', 'channel_mobile', 'channel_social', 'channel_email']
X = df[features]
y = df['target']

# Criar lista de tamanhos dos grupos
groups = df.groupby('group_id').size().tolist()

# Divisão treino-teste
X_train, X_test, y_train, y_test, groups_train, groups_test = train_test_split(
    X, y, groups, test_size=0.2, random_state=42
)

# Treinar LGBMRanker
ranker = lgb.LGBMRanker(
    objective='lambdarank',
    metric='ndcg',
    boosting_type='gbdt',
    n_estimators=100,
    learning_rate=0.1
)

ranker.fit(
    X_train, y_train, group=groups_train,
    eval_set=[(X_test, y_test)],
    eval_group=[groups_test],
    eval_at=[1, 3, 5], verbose=10
)

# Fazer predições e criar ranking
df['rank'] = ranker.predict(X)
df_resultados = df.sort_values(by=['account_id', 'rank'], ascending=[True, False])

# Exibir as top recomendações para cada cliente
df_top_recomendacoes = df_resultados.groupby('account_id').head(1)

print("Top Recomendações por Cliente:")
print(df_top_recomendacoes)
